# 🧠 12\_Extending\_Pydantic → 3\_Core\_Schema\_Hooks

### 🎯 Intent

Use **core schema hooks** to define **first-class custom types** in Pydantic v2—controlling validation & serialization at the **type level** (fast, reusable, library-grade).

---

### 🧩 Core Components / Functions

1. **🧩 `__get_pydantic_core_schema__` (Type Hook)**

   * Implement on your custom class to return a **core\_schema** that tells Pydantic how to **validate & serialize** the type.
   * Signature:

     ```python
     def __get_pydantic_core_schema__(cls, source, handler: GetCoreSchemaHandler) -> core_schema.CoreSchema: ...
     ```

2. **🔬 Validator Functions**

   * Use `core_schema.no_info_after_validator_function(fn, inner_schema)` or `with_info...` to wrap a base schema (e.g., `str_schema()`, `int_schema()`), then apply custom checks/coercion.
   * Pattern:

     ```python
     def validate(v):
         # normalize + check; raise ValueError/PydanticCustomError on fail
         return converted_value
     schema = core_schema.no_info_after_validator_function(validate, core_schema.str_schema())
     ```

3. **🧰 Serializer Hooks**

   * Attach a serializer so dumps work in both modes:

     * `core_schema.plain_serializer_function_ser_schema(fn, json_return_type='str'|'int'|'list'|...)`
   * Ensure JSON mode returns **JSON-safe** values (strings/numbers).

4. **🏗️ Composition with Handlers**

   * `handler(str)` or `handler.generate_schema(OtherType)` lets you **reuse** existing schemas, then add your layer.
   * Useful for **wrapping** built-ins or composing from other custom types.

5. **🧱 Example: ISO Country Code**

   * Base on `str_schema()` → uppercase + regex `^[A-Z]{2}$` in validator, serialize as plain string.
   * Be explicit with error via `PydanticCustomError("country_invalid", "invalid ISO-3166 alpha-2")`.

6. **♻️ Reusability & Speed**

   * Once defined, the type is just another field annotation everywhere (`CountryCode`), with **fast** core validation and clean schema output.

7. **📜 JSON Schema Integration**

   * Provide titles/formats via `json_schema_extra` in a **wrapper model** or implement `__get_pydantic_json_schema__` (advanced) to tweak schema generation for the type.

8. **🧪 Testing Strategy**

   * Unit test: valid/invalid samples; serialization round-trip in both modes (`mode="python"`, `"json"`).
   * Snapshot test the **JSON Schema** to lock contract.

9. **⚠️ Gotchas**

   * Keep validators **pure** (no I/O).
   * Return the **final typed value** (e.g., your subclass/normalized primitive).
   * Make sure serializer returns JSON-safe types in JSON mode; otherwise dumps may fail.

10. **🧠 When to Use Hooks vs. Simpler Options**

* Prefer **`Annotated[..., Field(...)]`** or **field/model validators** for app-level needs.
* Use **core schema hooks** for **library-quality** types reused across many models/projects.

---

